In [1]:
import numpy as np
import time

class Conv3x3:
    
    def __init__(self, num_filters):
        self.num_filters = num_filters
        self.filters = np.random.randn(num_filters, 3, 3) / 9
        #self.bias = np.random.randn(num_filters) / num_filters
        self.t_conv = 0
        self.t_back_prop = 0
        

    def iterate_regions(self, image):
        
        h, w = image.shape

        for i in range(h - 2):
              for j in range(w - 2):
                im_region = image[i:(i + 3), j:(j + 3)]
                yield im_region, i, j

    def forward(self, input):
        
        self.last_input = input

        h, w = input.shape
        
        
        output = np.zeros((h - 2, w - 2, self.num_filters))
        
        t1 = time.time()

        for i in range(h - 2):
              for j in range(w - 2):
                    
                    im_region = input[i:(i + 3), j:(j + 3)]
                    
                    for k in range(self.num_filters):
                        for m in range(3):
                            for n in range(3):
                                output[i, j,k] += im_region[m,n]*self.filters[k,m,n]
                                
                        output[i, j,k] = np.maximum(0,output[i, j,k])
                    
                            
        t2 = time.time()
        
        self.t_conv += t2 - t1
        
        return output

    def backprop(self, d_L_d_out, learn_rate):
        
        
        
        d_L_d_filters = np.zeros(self.filters.shape)
        
        u = time.time()

        for im_region, i, j in self.iterate_regions(self.last_input):
              for f in range(self.num_filters):
                d_L_d_filters[f] += d_L_d_out[i, j, f] * im_region

        self.filters -= learn_rate * d_L_d_filters

#         d_L_d_b = np.zeros(self.bias.shape)
        
#         for f in range(self.num_filters):
#                 d_L_d_b[f] += np.sum(d_L_d_out[:, :, f])
#         self.bias -= learn_rate * d_L_d_b
        
        v = time.time()
        
        self.t_back_prop += v-u
        
        
        return None




In [2]:
import numpy as np

class MaxPool2:
    
    def iterate_regions(self, image):
        h, w, _ = image.shape
        new_h = h // 2
        new_w = w // 2
    
        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                yield im_region, i, j
    
    def forward(self, input):
    
        self.last_input = input
        self.t_for_prop = 0
        self.t_back_prop = 0
        
        t1 = time.time()
        h, w, num_filters = input.shape
        output = np.zeros((h // 2, w // 2, num_filters))

        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.amax(im_region, axis=(0, 1))
        t2 = time.time()
        
        self.t_for_prop += t2 -t1

        return output

    def backprop(self, d_L_d_out):
       
        d_L_d_input = np.zeros(self.last_input.shape)

        t3 = time.time()
        for im_region, i, j in self.iterate_regions(self.last_input):
            h, w, f = im_region.shape
            amax = np.amax(im_region, axis=(0, 1))

            for i2 in range(h):
                for j2 in range(w):
                      for f2 in range(f):
                        # If this pixel was the max value, copy the gradient to it.
                            if im_region[i2, j2, f2] == amax[f2]:
                                  d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]
                                    
        
        d_L_d_input[self.last_input<0] = 0
        
        t4 = time.time()
        
        self.t_back_prop += t4 - t3

        return d_L_d_input

In [3]:
import numpy as np

class Softmax:

    def __init__(self, input_len, nodes):
        self.weights = np.random.randn(input_len, nodes) / input_len
        self.biases = np.zeros(nodes)
        self.t_for_prop = 0
        self.t_back_prop = 0

    def forward(self, input):
        
        t1 = time.time()
        self.last_input_shape = input.shape

        input = input.flatten()
        self.last_input = input

        input_len, nodes = self.weights.shape

        totals = np.dot(input, self.weights) + self.biases
        self.last_totals = totals

        exp = np.exp(totals)
        t2 = time.time()
        
        self.t_for_prop += t2 -t1
        
        return exp / np.sum(exp, axis=0)

    def backprop(self, d_L_d_out, learn_rate):
        
        t3 = time.time()
        for i, gradient in enumerate(d_L_d_out):
            if gradient == 0:
                continue

            t_exp = np.exp(self.last_totals)

            S = np.sum(t_exp)

            d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
            d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)

            d_t_d_w = self.last_input
            d_t_d_b = 1
            d_t_d_inputs = self.weights

            d_L_d_t = gradient * d_out_d_t

            d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
            d_L_d_b = d_L_d_t * d_t_d_b
            d_L_d_inputs = d_t_d_inputs @ d_L_d_t

            self.weights -= learn_rate * d_L_d_w
            self.biases -= learn_rate * d_L_d_b
            t4 = time.time()
            
            self.t_back_prop += t4 - t3
            
            return d_L_d_inputs.reshape(self.last_input_shape)

In [5]:
from mnist import MNIST

mndata = MNIST(r'C:\Users\krish\OneDrive\Documents\krishna\sem6\DL\mnist')

mndata.gz = True

train_images = np.zeros([400,28,28])
test_images = np.zeros([100,28,28])


images_tr,train_labels = mndata.load_training()
images_tr = np.array(images_tr[:400])
train_labels = np.array(train_labels[:400])
    

images_ts1,test_labels = mndata.load_testing()
images_ts = np.array(images_ts1[:100])
test_labels = np.array(test_labels[0:100])


for i in range(400):
    train_images[i] = images_tr[i].reshape(28,28) 
    
for i in range(100):
    test_images[i] = images_ts[i].reshape(28,28)
    
#print(train_images[0])

In [6]:
import mnist
import numpy as np
import time
import tracemalloc


conv = Conv3x3(8)
pool = MaxPool2()                  
softmax = Softmax(13 * 13 * 8, 10) 

def forward(image, label): 
    
    out = conv.forward((image / 255) )
    out = pool.forward(out)
    out = softmax.forward(out)
    
    
    loss = -np.log(out[label])
    acc = 1 if np.argmax(out) == label else 0

    return out, loss, acc

def train(im, label, lr=.005):
   
    out, loss, acc = forward(im, label)

    gradient = np.zeros(10)
    gradient[label] = -1 / out[label]

    gradient = softmax.backprop(gradient, lr)
    gradient = pool.backprop(gradient)
    gradient = conv.backprop(gradient, lr)
    
    return loss, acc


t1 = time.time()



#tracemalloc.start()

for epoch in range(3):
    print('--- Epoch %d ---' % (epoch + 1))

    permutation = np.random.permutation(len(train_images))
    train_images = train_images[permutation]
    train_labels = train_labels[permutation]
    
    loss = 0
    num_correct = 0
    for i, (im, label) in enumerate(zip(train_images, train_labels)):
        l, acc = train(im, label)
        loss += l
        num_correct += acc
        
    print('after epoch' + str(epoch+1) + 'loss is ' + str(loss/1000) + 'accuracy is' + str(num_correct/1000))

#my_complex_analysis_method()

#current, peak = tracemalloc.get_traced_memory()

# print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
# tracemalloc.stop()
    

t3 = time.time()

print('Testing on test data set')
loss = 0
num_correct = 0
for im, label in zip(test_images, test_labels):
    _, l, acc = forward(im, label)
    loss += l
    num_correct += acc

t4 = time.time()
    
    
num_tests = len(test_images)
print('Loss on test dataset:', loss / num_tests)
print(' Accuracy on test dataset:', num_correct / num_tests)

t2 = time.time()

print('time taken is :' , t2 -t1 , 'second')
print('time taken for forward prop ',conv.t_conv, 'seconds')
print('time taken for backward prop ',conv.t_back_prop, 'seconds')
# print(pool.t_for_prop)
# print(pool.t_back_prop)
# print(softmax.t_for_prop)
# print(softmax.t_back_prop)
# print('time taken for testing :' , t4 -t3)


--- Epoch 1 ---
after epoch1loss is 0.6736595332195036accuracy is0.203
--- Epoch 2 ---
after epoch2loss is 0.2565427746671555accuracy is0.321
--- Epoch 3 ---
after epoch3loss is 0.16720483317246174accuracy is0.344
Testing on test data set
Loss on test dataset: 0.5657081007693968
 Accuracy on test dataset: 0.84
time taken is : 151.80177187919617 second
time taken for forward prop  103.12785077095032 seconds
time taken for backward prop  39.06708598136902 seconds


# Ignore the blocks below

In [ ]:
from random import randint
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, show

def predict_out(image):
    
    out = conv.forward((image / 255) )
    out = pool.forward(out)
    out = softmax.forward(out)
    
    return np.argmax(out)



print("test cases: \n\n ")

for j in range(7):
    rand = randint(1000,2000)
    print("\n\ninput image :")
    test_img = np.array(images_ts1[rand]).reshape(28,28)
    imshow(test_img)
    show()
    
    print(" predicted Output" , end = "")
    print(predict_out(test_img))
    
    
    

In [ ]:
in_array = [1, 2, 3.78, 4, 5.54, 6.6, 7, 8 ] 
print ("Input array : ", in_array) 
  
out_array = np.clip(in_array, a_min = 2, a_max = 6) 
print ("Output array : ", out_array) 


In [ ]:
import math
f = np.random.randn(8, 3, 3) / 9
#print(f)

print('\n')

t3 = time.time()

for i in range(5):
    for i in range(1000):
        
        for i in range(len(f)):
            #print('\n')
            #print(f[i])
            #print('\n')
            for _ in range(4): 

                min = math.inf
                for j in range(len(f[i])):
                    for k in range(len(f[i][j])):
                        if abs(f[i,j,k]) < min:
                            min = abs(f[i,j,k])
                            min_j = j
                            min_k = k

                f[i][min_j][min_k] = math.inf

            for j in range(len(f[i])):
                for k in range(len(f[i][j])):
                    if f[i,j,k] == math.inf:
                        f[i,j,k] = 0



        for i in range(len(f)):

            for j in range(len(f[i])):

                for k in range(len(f[i][j])):

                    if f[i,j,k] > 0 :
                        f[i,j,k] = 1

                    if f[i,j,k] < 0 :
                        f[i,j,k] = -1
                        

for i in range(1000):
        
        for i in range(len(f)):
            #print('\n')
            #print(f[i])
            #print('\n')
            for _ in range(4): 

                min = math.inf
                for j in range(len(f[i])):
                    for k in range(len(f[i][j])):
                        if abs(f[i,j,k]) < min:
                            min = abs(f[i,j,k])
                            min_j = j
                            min_k = k

                f[i][min_j][min_k] = math.inf

            for j in range(len(f[i])):
                for k in range(len(f[i][j])):
                    if f[i,j,k] == math.inf:
                        f[i,j,k] = 0



        for i in range(len(f)):

            for j in range(len(f[i])):

                for k in range(len(f[i][j])):

                    if f[i,j,k] > 0 :
                        f[i,j,k] = 1

                    if f[i,j,k] < 0 :
                        f[i,j,k] = -1
            

t4 = time.time()

print(t4 - t3)
            
    
            

In [ ]:
tracemalloc.start()


#my_complex_analysis_method()
current, peak = tracemalloc.get_traced_memory()

print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
tracemalloc.stop()
